In [1]:
username = "pgangan@getindigo.com"
password = "gPradyuman@02"
cloudProvider = "AWS"

In [7]:
!pip install pandas

In [2]:
import pandas as pd

In [3]:
from closedloop.api import *
saveClosedLoopCredentials(username, password, cloudProvider)

In [4]:
cl = ClosedLoopClient()

In [5]:
psId = 'rubicon_founders'

In [14]:
popSteps = [
            "2017-2021 Current Training States",
            LimitPercent(name = "Limit", percent=0.1)
        ]

features = [

    #Feature(
    #        name="Indigo Specialty",
    #        description="A medical practitioner's designated Indigo Specialty, from the Indigo Specialty Crosswalk.",
    #        clExpression="{Indigo_Specialties,select=[relativity_indigo_specialty],agg=[Last], asOfDate<=[P0]}",
    #    ),

    #Feature(
    #        name="Repair Procedures for Thoracic Aortic Aneurysm",
    #        description="",
    #        clExpression="{Procedures,select=[number_procedures], procedure=[CPT:4_33858_33877], date=[P-1Y:P0],agg=[Last]}",
    #    ),
    
     FeatureGenerator(
             name="Count of Procedures (CCS PCS Level-3) (1Y)",

             clExpression="{Procedures,select=[number_procedures],date=[P-1Y:P0],agg=[Last]}",
             featureNameTemplate="Count of procedures in category: <ontology_description>",
             featureDescriptionTemplate="Total count of procedures performed in category <ontology_description>",
             generators=[
                 FilterGeneratorOntology(
                     name="procedure",
                     rootCode="ROOT",
                     relationshipType="InSource",
                     source="CCSPCS",
                     level=3
                 ),
             ],
         ),


]
# level 2 CCSPCS has 206 features
# level 3 CCSPCS has 340 features
# Level 2 CPT has 768  features

df_check = dfQuery(cl, psId, features, popSteps)
display(df_check.shape)
display(df_check)

(542529, 339)

Count of procedures in category_ Magnetic resonance imaging  \
npi        indexTime                                                                 
1003000159 2017-01-01                                                  0             
           2018-01-01                                                  0             
           2019-01-01                                                  0             
           2020-01-01                                                  0             
           2021-01-01                                                  0             
...                                                                  ...             
1992999585 2017-01-01                                                  0             
           2018-01-01                                                  0             
           2019-01-01                                                  0             
           2020-01-01                                                  0             
           2021-01-01                                                  0             

                       Count of procedures in category_ Open cholecystectomy  \
npi        indexTime                                                           
1003000159 2017-01-01                                                  0       
           2018-01-01                                                  0       
           2019-01-01                                                  0       
           2020-01-01                                                  0       
           2021-01-01                                                  0       
...                                                                  ...       
1992999585 2017-01-01                                                  0       
           2018-01-01                                                  0       
           2019-01-01                                                  0       
           2020-01-01                                                  0       
           2021-01-01                                                  0       

                       Count of procedures in category_ Non-OR procedures on nasal sinuses  \
npi        indexTime                                                                         
1003000159 2017-01-01                                                  0                     
           2018-01-01                                                  0                     
           2019-01-01                                                  0                     
           2020-01-01                                                  0                     
           2021-01-01                                                  0                     
...                                                                  ...                     
1992999585 2017-01-01                                                  0                     
           2018-01-01                                                  0                     
           2019-01-01                                                  0                     
           2020-01-01                                                  0                     
           2021-01-01                                                  0                     

                       Count of procedures in category_ Bronchoscopy without biopsy  \
npi        indexTime                                                                  
1003000159 2017-01-01                                                  0              
           2018-01-01                                                  0              
           2019-01-01                                                  0              
           2020-01-01                                                  0              
           2021-01-01                                                  0              
...                            

In [15]:
df_check = df_check.reset_index()

In [16]:
df_check.drop(columns=['indexTime'], inplace=True, errors='ignore')
df_check = df_check.groupby('npi').sum().reset_index()

In [17]:
df_check

,npi,Count of procedures in category_ Magnetic resonance imaging,Count of procedures in category_ Open cholecystectomy,Count of procedures in category_ Non-OR procedures on nasal sinuses,Count of procedures in category_ Bronchoscopy without biopsy,Count of procedures in category_ Other respiratory treatments,Count of procedures in category_ Incision/excision of anal fistula,Count of procedures in category_ Other non-OR therapeutic procedures_ female organs,Count of procedures in category_ Repair of retinal tear_ detachment,Count of procedures in category_ Creation of cutaneoperitoneal fistula,...,Count of procedures in category_ Other forceps or breech delivery,Count of procedures in category_ Bilateral orchiectomy,Count of procedures in category_ Electroencephalogram _EEG,Count of procedures in category_ Other muscle and tendon procedures,Count of procedures in category_ Nonoperative removal of foreign body,Count of procedures in category_ Laparoscopic cholecystectomy,Count of procedures in category_ Other operations on ovary,Count of procedures in category_ Oxygen therapy,Count of procedures in category_ Cesarean section,Count of procedures in category_ Upper gastrointestinal X-ray
0,1003000159,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1003000621,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1003001462,6,0,0,0,0,0,15,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1003002064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1003002205,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121696,1992997076,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121697,1992998090,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
121698,1992998181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0,0,0,0,0
121699,1992998926,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_check.to_pickle('./ccspcs_npi_features.pkl')

In [23]:
for _, r in df_check.iterrows():
    for val in r:
        if int(val) > 0:
            print(val)
    break

1003000159


In [6]:
def save_table(cl, psId, sql, save_location):
    df = sqlQuery(cl=cl, psId=psId, sql=sql)
    df.to_pickle(save_location)

In [7]:
sql = '''
    SELECT * 
    FROM Physicians_Commercial_Procedures_HCPCS_CPT_2019
 '''

save_location = './procedures_aggregation_2019.pkl'

In [17]:
save_table(cl, psId, sql, save_location)

In [18]:
df = pd.read_pickle(save_location)
df.head()

,NPI,FIRST_NAME,LAST_NAME,CLAIM_YEAR,HCPCS_CPT_CODE,HCPCS_CPT_DESCRIPTION,NUMBER_PROCEDURES,NUMBER_UNIQUE_PATIENTS,TOTAL_CHARGES,AVG_CHARGE_PER_PROCEDURE,HCPCS_CPT_CATEGORY,HCPCS_CPT_SUBCATEGORY
0,1003063181,Anita,Nagamine,2020,19083,Bx breast 1st lesion us imag,1,1,2098.0,2098.000000,Surgery,Surgery - Integumentary System\r
1,1003063181,Anita,Nagamine,2020,38505,NEEDLE BIOPSY LYMPH NODES,1,1,414.0,414.000000,Surgery,Surgery - Hemic and Lymphatic Systems\r
2,1003063181,Anita,Nagamine,2020,76641,ULTRASOUND BREAST COMPLETE,26,26,4666.0,179.461538,Radiology,Radiology - Diagnostic Ultrasound\r
3,1003063181,Anita,Nagamine,2020,76642,ULTRASOUND BREAST LIMITED,21,21,2715.0,129.285714,Radiology,Radiology - Diagnostic Ultrasound\r
4,1003063181,Anita,Nagamine,2020,76882,US LMTD JT/NONVASC XTR STRUX,4,3,312.0,78.000000,Radiology,Radiology - Diagnostic Ultrasound\r


In [20]:
len_ = 0

for _, r in df.iterrows():
    len_+=1

print(len_)

93305083


In [21]:
sql = '''
    SELECT * 
    FROM indigo_specialty_to_nucc_crosswalk
 '''

save_location = './indigo_speciality_to_nucc_crosswalk.pkl'

In [22]:
save_table(cl, psId, sql, save_location)

Reauthenticating web socket call


In [23]:
crosswalk = pd.read_pickle(save_location)

In [24]:
crosswalk.head()

,NUCC Display Name,NUCC Classification,NUCC Code,Indigo Specialty,OLD Indigo Class Specialty
0,Sports Medicine (Internal Medicine) Physician,Internal Medicine,207RS0010X,Internal Medicine-No Surgery,Internal Medicine - Non-Surgical
1,Pediatric Dentist,Dentist,1223P0221X,NaN,NaN
2,Optometrist,Optometrist,152W00000X,Optometrist,NaN
3,Pediatric Rehabilitation Medicine Physician,Physical Medicine & Rehabilitation,2081P0010X,Physiatry,Physiatry - Non-Surgical
4,Radiology Podiatrist,Podiatrist,213ER0200X,Podiatrist,Podiatry
